<a href="https://colab.research.google.com/github/Rohanjain1211/UML501_Rohan_Jain_102303740/blob/main/assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder, KBinsDiscretizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import jaccard_score, pairwise_distances
from scipy.spatial.distance import cosine
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

q1


In [26]:
data_cus=pd.read_csv('/content/AWCustomers.csv')
data_sales=pd.read_csv('/content/AWSales.csv')

In [27]:
data_cus.head()

,CustomerID,Title,FirstName,MiddleName,LastName,Suffix,AddressLine1,AddressLine2,City,StateProvinceName,...,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,LastUpdated
0,21173,NaN,Chad,C,Yuan,NaN,7090 C. Mount Hood,NaN,Wollongong,New South Wales,...,Bachelors,Clerical,M,M,1,3,0,1,81916,2017-03-06
1,13249,NaN,Ryan,NaN,Perry,NaN,3651 Willow Lake Rd,NaN,Shawnee,British Columbia,...,Partial College,Clerical,M,M,1,2,1,2,81076,2017-03-06
2,29350,NaN,Julia,NaN,Thompson,NaN,1774 Tice Valley Blvd.,NaN,West Covina,California,...,Bachelors,Clerical,F,S,0,3,0,0,86387,2017-03-06
3,13503,NaN,Theodore,NaN,Gomez,NaN,2103 Baldwin Dr,NaN,Liverpool,England,...,Partial College,Skilled Manual,M,M,1,2,1,2,61481,2017-03-06
4,22803,NaN,Marshall,J,Shan,NaN,Am Gallberg 234,NaN,Werne,Nordrhein-Westfalen,...,Partial College,Skilled Manual,M,S,1,1,0,0,51804,2017-03-06


In [28]:
data_sales.columns

Index(['CustomerID', 'BikeBuyer', 'AvgMonthSpend'], dtype='object')

In [29]:
data_sales.drop(['CustomerID'],axis=1,inplace=True)

In [30]:
df=pd.concat([data_cus,data_sales],axis=1)

In [31]:
df.head()

,CustomerID,Title,FirstName,MiddleName,LastName,Suffix,AddressLine1,AddressLine2,City,StateProvinceName,...,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,LastUpdated,BikeBuyer,AvgMonthSpend
0,21173,NaN,Chad,C,Yuan,NaN,7090 C. Mount Hood,NaN,Wollongong,New South Wales,...,M,M,1,3,0,1,81916,2017-03-06,1.0,50.97
1,13249,NaN,Ryan,NaN,Perry,NaN,3651 Willow Lake Rd,NaN,Shawnee,British Columbia,...,M,M,1,2,1,2,81076,2017-03-06,1.0,53.11
2,29350,NaN,Julia,NaN,Thompson,NaN,1774 Tice Valley Blvd.,NaN,West Covina,California,...,F,S,0,3,0,0,86387,2017-03-06,1.0,54.08
3,13503,NaN,Theodore,NaN,Gomez,NaN,2103 Baldwin Dr,NaN,Liverpool,England,...,M,M,1,2,1,2,61481,2017-03-06,1.0,56.93
4,22803,NaN,Marshall,J,Shan,NaN,Am Gallberg 234,NaN,Werne,Nordrhein-Westfalen,...,M,S,1,1,0,0,51804,2017-03-06,1.0,55.41


In [32]:
df.drop(['Title','Suffix','Education','Occupation','PhoneNumber','MiddleName','AddressLine2'],axis=1,inplace=True)

In [33]:
df.columns

Index(['CustomerID', 'FirstName', 'LastName', 'AddressLine1', 'City',
       'StateProvinceName', 'CountryRegionName', 'PostalCode', 'BirthDate',
       'Gender', 'MaritalStatus', 'HomeOwnerFlag', 'NumberCarsOwned',
       'NumberChildrenAtHome', 'TotalChildren', 'YearlyIncome', 'LastUpdated',
       'BikeBuyer', 'AvgMonthSpend'],
      dtype='object')

In [34]:
df.describe()

,CustomerID,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,BikeBuyer,AvgMonthSpend
count,18361.000000,18361.000000,18361.000000,18361.000000,18361.000000,18361.000000,18355.00000,18355.000000
mean,20241.987092,0.610588,1.270301,0.338162,0.850389,72754.779642,0.55173,51.767207
std,5336.332731,0.487630,0.913989,0.568957,0.927315,30686.014313,0.49733,3.438024
min,11000.000000,0.000000,0.000000,0.000000,0.000000,25435.000000,0.00000,44.100000
25%,15621.000000,0.000000,1.000000,0.000000,0.000000,53312.000000,0.00000,49.410000
50%,20248.000000,1.000000,1.000000,0.000000,0.000000,61851.000000,1.00000,51.420000
75%,24863.000000,1.000000,2.000000,1.000000,2.000000,87410.000000,1.00000,53.600000
max,29483.000000,1.000000,5.000000,3.000000,3.000000,139115.000000,1.00000,65.290000


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18361 entries, 0 to 18360
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   CustomerID            18361 non-null  int64  
 1   FirstName             18361 non-null  object 
 2   LastName              18361 non-null  object 
 3   AddressLine1          18361 non-null  object 
 4   City                  18361 non-null  object 
 5   StateProvinceName     18361 non-null  object 
 6   CountryRegionName     18361 non-null  object 
 7   PostalCode            18361 non-null  object 
 8   BirthDate             18361 non-null  object 
 9   Gender                18361 non-null  object 
 10  MaritalStatus         18361 non-null  object 
 11  HomeOwnerFlag         18361 non-null  int64  
 12  NumberCarsOwned       18361 non-null  int64  
 13  NumberChildrenAtHome  18361 non-null  int64  
 14  TotalChildren         18361 non-null  int64  
 15  YearlyIncome       

In [36]:
var_cat=[]
var_num=[]
for c in df.columns:
    if df[c].dtype=='float64':
        var_cat.append(c)
    if df[c].dtype=='int64':
         var_num.append(c)

print("Discrete Variables: ",var_num)
print("Continuous variables: ",var_cat)

Discrete Variables:  ['CustomerID', 'HomeOwnerFlag', 'NumberCarsOwned', 'NumberChildrenAtHome', 'TotalChildren', 'YearlyIncome']
Continuous variables:  ['BikeBuyer', 'AvgMonthSpend']


In [37]:
Nominal = [
    'CustomerID',
    'FirstName',
    'LastName',
    'AddressLine1',
    'City',
    'StateProvinceName',
    'CountryRegionName',
    'PostalCode',
    'Gender',
    'MaritalStatus',
    'HomeOwnerFlag',
    'BikeBuyer'
]
Ordinal=[]
Ratio = [
    'NumberCarsOwned',
    'NumberChildrenAtHome',
    'TotalChildren',
    'YearlyIncome',
    'AvgMonthSpend'
]
Interval=['BirthDate']
print("Nominal: ",Nominal)
print("Ordinal: ",Ordinal)
print("Ratio: ",Ratio)
print("Interval: ",Interval)

Nominal:  ['CustomerID', 'FirstName', 'LastName', 'AddressLine1', 'City', 'StateProvinceName', 'CountryRegionName', 'PostalCode', 'Gender', 'MaritalStatus', 'HomeOwnerFlag', 'BikeBuyer']
Ordinal:  []
Ratio:  ['NumberCarsOwned', 'NumberChildrenAtHome', 'TotalChildren', 'YearlyIncome', 'AvgMonthSpend']
Interval:  ['BirthDate']


Q2

In [38]:
numerical_features = ['YearlyIncome', 'AvgMonthSpend', 'NumberCarsOwned', 'NumberChildrenAtHome', 'TotalChildren']
categorical_features = ['Gender', 'MaritalStatus', 'HomeOwnerFlag', 'BikeBuyer']

numerical_imputer = SimpleImputer(strategy='mean')
categorical_imputer = SimpleImputer(strategy='most_frequent')

df[numerical_features] = numerical_imputer.fit_transform(df[numerical_features])
df[categorical_features] = categorical_imputer.fit_transform(df[categorical_features])

normalized


In [39]:
scaler = MinMaxScaler()
df[numerical_features] = scaler.fit_transform(df[numerical_features])

Discretization (Binning)

In [40]:
binning_transformer = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='uniform')
df['YearlyIncome_Binned'] = binning_transformer.fit_transform(df[['YearlyIncome']])

Strandardization

In [41]:
standard_scaler = StandardScaler()
df[numerical_features] = standard_scaler.fit_transform(df[numerical_features])

In [42]:
encoder = OneHotEncoder()
encoded_categorical_features = encoder.fit_transform(df[categorical_features]).toarray()
encoded_df = pd.DataFrame(encoded_categorical_features, columns=encoder.get_feature_names_out(categorical_features))
df = pd.concat([df, encoded_df], axis=1)

Q3


In [43]:
def jaccard(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union


In [44]:
jaccard(df['YearlyIncome'],df['AvgMonthSpend'])

0.0

In [46]:
df['NumberChildrenAtHome_Binary'] = (df['NumberChildrenAtHome'] > 0).astype(int)

for bike_buyer_col in ['BikeBuyer_0.0', 'BikeBuyer_1.0']:
    jaccard_sim = jaccard_score(df['NumberChildrenAtHome_Binary'], df[bike_buyer_col])
    print(f"Jaccard Similarity between NumberChildrenAtHome_Binary and {bike_buyer_col}: {jaccard_sim}")

Jaccard Similarity between NumberChildrenAtHome_Binary and BikeBuyer_0.0: 0.2138122536725188
Jaccard Similarity between NumberChildrenAtHome_Binary and BikeBuyer_1.0: 0.23450479233226837


In [47]:
df['YearlyIncome'].corr(df['AvgMonthSpend'])

np.float64(0.012200386558915565)